# Práctico 2: Recomendación de videojuegos

Autor: Mariano Zibecchi  
mzibecchi@gmail.com . 

La resolucion de la primer parte del practico, consiste en la definicion de una funcion __recomend_games_for_user__

La funcion recibe como parametros el username, el modelo entrenado y la cantidad de recomendaciones que se desea recibir.

Retorna como resultado una lista de juegos recomendados.
  
  
  
  

En este práctico trabajaremos con un subconjunto de datos sobre [videojuegos de Steam](http://cseweb.ucsd.edu/~jmcauley/datasets.html#steam_data). Para facilitar un poco el práctico, se les dará el conjunto de datos previamente procesado. En este mismo notebook mostraremos el proceso de limpieza, para que quede registro del proceso (de todas maneras, por el tamaño de los datos no recomendamos que pierdan tiempo en el proceso salvo que lo consideren útil a fines personales). 

El conjunto de datos se basa en dos partes: lista de juegos (items), y lista de reviews de usuarios sobre distintos juegos. Este último, en su versión original es muy grande, (pesa 1.3GB), por lo que será solo una muestra del mismo sobre la que trabajarán.

A diferencia del conjunto de datos de LastFM utilizados en el [Práctico 1](./practico1.ipynb), en este caso los datos no están particularmente pensados para un sistema de recomendación, por lo que requerirá de un poco más de trabajo general sobre el dataset.

La idea es que, de manera similar al práctico anterior, realicen un sistema de recomendación. A diferencia del práctico anterior, este será un poco más completo y deberán hacer dos sistemas, uno que, dado un nombre de usuario le recomiende una lista de juegos, y otro que dado el título de un juego, recomiende una lista de juegos similares. Además, en este caso se requiere que el segundo sistema (el que recomienda juegos basado en el nombre de un juego en particular) haga uso de la información de contenido (i.e. o bien harán un filtrado basado en contenido o algo híbrido).

## Obtención y limpieza del conjunto de datos

El conjunto de datos originalmente se encuentra en archivos que deberían ser de formato "JSON". Sin embargo, en realidad es un archivo donde cada línea es un objeto de JSON. Hay un problema no obstante y es que las líneas están mal formateadas, dado que no respetan el estándar JSON de utilizar comillas dobles (**"**) y en su lugar utilizan comillas simples (**'**). Afortunadamente, se pueden evaluar como diccionarios de Python, lo cuál permite trabajarlos directamente.

## Ejercicio 1: Análisis Exploratorio de Datos

Ya teniendo los datos, podemos cargarlos y empezar con el práctico. Antes que nada vamos a hacer una exploración de los datos. Lo principal a tener en cuenta para este caso es que debemos identificar las variables con las que vamos a trabajar. A diferencia del práctico anterior, este conjunto de datos no está documentado, por lo que la exploración es necesaria para poder entender que cosas van a definir nuestro sistema de recomendación.

In [1]:
import pandas as pd

### Características del conjunto de datos sobre videojuegos

Las características del conjunto de datos de videojuegos tienen la información necesaria para hacer el "vector de contenido" utilizado en el segundo sistema de recomendación. Su tarea es hacer un análisis sobre dicho conjunto de datos y descartar aquella información redundante.

In [2]:
games_df = pd.read_json("./data/steam/games.json.gz")
games_df.sample(15)

,publisher,genres,app_name,title,release_date,tags,discount_price,specs,price,early_access,id,developer,sentiment,metascore
2526,Pictopotamus,"[Design &amp; Illustration, Photo Editing]",Pixeluvo,Pixeluvo,2014-08-14,"[Photo Editing, Design & Illustration]",NaN,None,34.99,False,314500.0,Pictopotamus,Very Positive,None
14769,Energize Labs Inc.,"[Action, Indie]",Epsilon Jump Prime,Epsilon Jump Prime,2017-10-20,"[Action, Indie]",NaN,"[Single-player, Partial Controller Support]",3.99,False,600230.0,Energize Labs Inc.,None,None
17763,Rebellion,[Action],Battlezone - Classic Camo (Skin),Battlezone - Classic Camo (Skin),2017-05-11,[Action],NaN,"[Single-player, Online Multi-Player, Online Co...",0.99,False,594450.0,Rebellion,None,None
17018,Tusky Games,"[Action, Adventure, Indie]",Escape From BioStation,Escape From BioStation,2017-07-25,"[Action, Indie, Adventure]",NaN,"[Single-player, Steam Achievements, Steam Trad...",2.99,False,391650.0,Tusky Games,2 user reviews,None
29513,United Independent Entertainment GmbH,"[Adventure, Indie, RPG]",Realms of Arkania: Blade of Destiny - For the ...,Realms of Arkania: Blade of Destiny - For the ...,2014-02-07,"[RPG, Adventure, Indie]",NaN,"[Single-player, Downloadable Content]",2.99,False,264840.0,Crafty Studios,1 user reviews,None
18311,None,"[Adventure, Indie]",White Noise 2 - Supporter Pack,White Noise 2 - Supporter Pack,2017-04-06,"[Adventure, Indie, Horror, Survival Horror, On...",NaN,"[Single-player, Multi-player, Online Multi-Pla...",1.99,False,610520.0,Milkstone Studios,Positive,None
11598,None,None,Ranma 1/2 OVA and Movie Collection,Ranma 1/2 OVA and Movie Collection,1989-04-05,[Episodic],NaN,None,2.99,False,713290.0,None,1 user reviews,None
31722,Three Donkeys LLC,[Strategy],Spectromancer,Spectromancer,2008-10-15,"[Strategy, Card Game]",NaN,"[Single-player, Multi-player, Online Multi-Pla...",9.99,False,22500.0,"Apus Software,Three Donkeys LLC",Very Positive,NA
13684,Tomáš Sabol,"[Adventure, Indie]",Hunter of antiques,Hunter of antiques,2017-11-30,"[Adventure, Indie]",NaN,"[Single-player, Steam Cloud]",9.99,False,745060.0,Tomáš Sabol,1 user reviews,None
22133,Zeiva Inc,"[Adventure, Casual, Indie]",Anicon - Animal Complex - Cat's Path,Anicon - Animal Complex - Cat's Path,2016-07-02,"[Adventure, Indie, Casual, Visual Novel, Otome...",NaN,"[Single-player, Steam Trading Cards]",7.99,False,502120.0,Zeiva Inc,Positive,None


In [3]:
len(games_df)

32135

In [4]:
games_df.dtypes

publisher          object
genres             object
app_name           object
title              object
release_date       object
tags               object
discount_price    float64
specs              object
price              object
early_access         bool
id                float64
developer          object
sentiment          object
metascore          object
dtype: object

In [5]:
games_df['id'].isnull().sum()

2

In [6]:
#games_df = games_df.dropna()
games_df[ games_df['id'].isnull() ]


,publisher,genres,app_name,title,release_date,tags,discount_price,specs,price,early_access,id,developer,sentiment,metascore
74,None,None,None,None,None,None,14.99,None,19.99,False,NaN,None,None,None
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)",Overwhelmingly Positive,91


In [7]:
games_df = games_df.dropna (  subset=['id'] )

In [8]:
games_df[ games_df['id'].isnull() ]

,publisher,genres,app_name,title,release_date,tags,discount_price,specs,price,early_access,id,developer,sentiment,metascore


In [9]:
games_df

,publisher,genres,app_name,title,release_date,tags,discount_price,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,[Single-player],4.99,False,761140.0,Kotoshiro,None,None
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980.0,Secret Level SRL,Mostly Positive,None
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290.0,Poolians.com,Mostly Positive,None
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",0.83,[Single-player],0.99,False,767400.0,彼岸领域,None,None
4,None,None,Log Challenge,None,None,"[Action, Indie, Casual, Sports]",1.79,"[Single-player, Full controller support, HTC V...",2.99,False,773570.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32130,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]",1.49,"[Single-player, Steam Achievements]",1.99,False,773640.0,"Nikita ""Ghost_RUS""",None,None
32131,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]",4.24,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530.0,Sacada,None,None
32132,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,2018-01-04,"[Indie, Simulation, Racing]",1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660.0,Laush Dmitriy Sergeevich,None,None
32133,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870.0,"xropi,stev3ns",1 user reviews,None


In [10]:
games_df['id'] = games_df['id'].astype(int)

In [11]:
games_df.head()

,publisher,genres,app_name,title,release_date,tags,discount_price,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,[Single-player],4.99,False,761140,Kotoshiro,None,None
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,None
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,None
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",0.83,[Single-player],0.99,False,767400,彼岸领域,None,None
4,None,None,Log Challenge,None,None,"[Action, Indie, Casual, Sports]",1.79,"[Single-player, Full controller support, HTC V...",2.99,False,773570,None,None,None


In [12]:
games_df.metascore.unique()

array([None, 96, 84, 80, 76, 70, 'NA', 69, 81, 75, 72, 66, 67, 77, 91, 89,
       83, 61, 88, 65, 94, 57, 86, 87, 92, 79, 82, 58, 74, 85, 90, 68, 71,
       60, 73, 59, 64, 54, 53, 78, 51, 44, 63, 38, 56, 49, 52, 62, 93, 48,
       34, 95, 43, 55, 24, 46, 41, 20, 39, 45, 35, 47, 40, 36, 50, 32, 37,
       33, 42, 27, 29, 30], dtype=object)

In [13]:
games_df.sentiment.unique()

array([None, 'Mostly Positive', 'Mixed', '1 user reviews',
       '3 user reviews', '8 user reviews', 'Very Positive',
       'Overwhelmingly Positive', '6 user reviews', '5 user reviews',
       '2 user reviews', 'Very Negative', 'Positive', 'Mostly Negative',
       '9 user reviews', 'Negative', '4 user reviews', '7 user reviews',
       'Overwhelmingly Negative'], dtype=object)

### Características del conjunto de datos de reviews

Este será el conjunto de datos a utilizar para obtener información sobre los usuarios y su interacción con videojuegos. Como se puede observar no hay un rating explícito, sino uno implícito a calcular, que será parte de su trabajo (deberán descubrir que característica les puede dar información que puede ser equivalente a un rating).

In [14]:
reviews_df = pd.read_json("./data/steam/reviews.json.gz")
reviews_df.sample(15)

,username,product_id,page_order,text,hours,products,date,early_access,page,compensation,found_funny,user_id
10608,HaaWingard!,206440,8,"This game is an amazing experience, with an in...",5.1,310.0,2015-04-05,False,901,None,NaN,NaN
316114,BrudnyRobert-Parapetiusz-,258970,3,"Little simple hack and slash, can be fun playi...",9.2,329.0,2016-11-13,False,59,None,NaN,NaN
5294,Smile Doge,293840,9,Very interesting concept for a game but lacks ...,0.3,537.0,2016-08-31,False,3,Product received for free,NaN,NaN
288208,TDLM,242050,7,"The best Assassin's Creed, period.\nI don't ev...",26.8,188.0,2017-12-12,False,14,None,1.0,7.656120e+16
94357,draziw,709920,9,Diabolical deterministic oddly-shaped mineswee...,54.5,355.0,2017-11-29,False,1,None,NaN,NaN
362584,DamnDude,323850,8,"5/5 game review, huh what was that\nArin told ...",2.7,45.0,2016-05-24,False,139,None,NaN,NaN
313762,Dodger7,444090,4,It's a good game. The game is not everyone's t...,5.2,40.0,2017-01-04,True,3109,None,NaN,NaN
250244,Real_warrior,281990,9,"Created race of repugnant, decadent, weak, sed...",612.7,319.0,2016-06-21,False,1009,None,3.0,7.656120e+16
572792,HaMMeR™,440,9,Half-Life 3 died for this.,19.1,3199.0,2017-08-29,False,1825,None,1.0,NaN
447565,BuZz_Parrallax,4000,4,i need people to play with please send friend ...,30.3,5.0,2016-12-27,False,3271,None,NaN,NaN


Descripcion del Dataset:  
* hours - horas jugadas antes de emitir el review
* early_access - si se trata de una version early access del producto
**Early Access is not meant to be a form of pre-purchase, but a tool to get your game in front of Steam users and gather feedback while finishing your game. Early Access titles must deliver a playable game or usable software to the customer at the time of purchase, while pre-purchase games are delivered at a future date.**
* found_funny - otros usuarios encontraron el review divertido
* compensation - 
* products - indica los diferentes juegos q tiene la cuenta en su libreria

In [15]:
user = 'TheDoofMoments'
reviews_df[ reviews_df['username'] == user ]

,username,product_id,page_order,text,hours,products,date,early_access,page,compensation,found_funny,user_id
127491,TheDoofMoments,200210,0,I like this game,625.3,65.0,2017-07-02,False,150,None,NaN,NaN


In [16]:
user = 'SPejsMan'
reviews_df[ reviews_df['username'] == user ]

,username,product_id,page_order,text,hours,products,date,early_access,page,compensation,found_funny,user_id
0,SPejsMan,227940,0,Just one word... Balance!,23.0,92.0,2015-02-25,True,3159,None,NaN,NaN
87241,SPejsMan,282070,7,It is good to be bad.,13.8,92.0,2016-11-26,False,459,None,NaN,NaN


In [17]:
user = 'Agron Bulchard-Chataeu'
reviews_df[ reviews_df['username'] == user ]

,username,product_id,page_order,text,hours,products,date,early_access,page,compensation,found_funny,user_id


Veamos un juego en particular __Total War™: ROME II - Emperor Edition__  
y los reviews que tuvo...  


In [18]:
games_df[ games_df ['id'] == 214950 ]

,publisher,genres,app_name,title,release_date,tags,discount_price,specs,price,early_access,id,developer,sentiment,metascore
30024,SEGA,[Strategy],Total War™: ROME II - Emperor Edition,Total War™: ROME II - Emperor Edition,2013-09-02,"[Strategy, Historical, Turn-Based Strategy, Gr...",NaN,"[Single-player, Multi-player, Steam Trading Ca...",59.95,False,214950,Creative Assembly,Mostly Positive,76


In [19]:
roma_reviews_df = reviews_df [ reviews_df['product_id'] == 214950]
roma_reviews_df

,username,product_id,page_order,text,hours,products,date,early_access,page,compensation,found_funny,user_id
7,CaptainPlanet,214950,1,"If you like slaughtering in the name of Rome, ...",203.3,274.0,2016-10-27,False,285,None,NaN,7.656120e+16
1449,Ciddie Fobbler,214950,5,Does ok at the things that are ok,130.6,115.0,2015-08-08,False,621,None,NaN,NaN
1896,turtleownage,214950,1,"Super fun, battles are epic and a treat to wat...",7.1,40.0,2017-10-25,False,78,None,NaN,7.656120e+16
2259,battlegalactika,214950,8,The combat is simply broken\nIt doesnt feel li...,76.9,5.0,2017-05-12,False,153,None,NaN,7.656120e+16
2468,Terminally Chill™,214950,8,Stiltzkin: are you high\nStiltzkin: youve been...,43.0,17.0,2015-02-15,False,816,None,10.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
697932,sosacharly30,214950,1,WORSE♥♥♥♥♥♥I HAVE EVER PLAYED BY CA I SWEAR SL...,805.1,23.0,2014-08-01,False,1195,None,NaN,7.656120e+16
698520,DrFDisK,214950,1,Worse than I expected. Still not a finished ga...,6.1,265.0,2015-06-28,False,664,None,NaN,NaN
699106,AMP2533,214950,1,I personally like this game but after having s...,96.4,95.0,2016-02-10,False,467,None,NaN,NaN
699216,Adjusted for Infeeltion,214950,4,A great strategy game built on the reliable to...,70.8,170.0,2016-11-23,False,266,None,NaN,NaN


In [20]:
user = 'sosacharly30'
reviews_df[ reviews_df['username'] == user ].text

697932    WORSE♥♥♥♥♥♥I HAVE EVER PLAYED BY CA I SWEAR SL...
Name: text, dtype: object

In [21]:
roma_reviews_df.username.value_counts()

Man Ray         2
REVAN123REX     2
tavin44         2
Ozymandias      2
Dank Meme       2
               ..
sta7ic          1
hrkisk          1
Typhoon         1
DaCaptain150    1
reeeee          1
Name: username, Length: 1228, dtype: int64

In [22]:
user = 'Caesar'
roma_reviews_df[ roma_reviews_df['username'] == user ]

,username,product_id,page_order,text,hours,products,date,early_access,page,compensation,found_funny,user_id
209383,Caesar,214950,4,can't compare to rome 1,686.2,36.0,2014-09-29,False,1081,None,NaN,NaN
647322,Caesar,214950,4,I love everything simply because of the mod ba...,495.3,86.0,2014-09-20,False,1117,None,NaN,NaN


In [23]:
len(reviews_df)

700000

In [24]:
reviews_df.describe()

,product_id,page_order,hours,products,page,found_funny,user_id
count,700000.000000,700000.000000,697558.000000,698708.000000,700000.000000,107375.000000,2.855870e+05
mean,251130.840463,4.485711,111.498797,236.192551,890.908570,7.738217,7.656120e+16
std,150044.054746,2.875279,385.359458,485.337170,1923.769739,71.931147,1.009730e+08
min,10.000000,0.000000,0.000000,1.000000,1.000000,1.000000,7.656120e+16
25%,203770.000000,2.000000,4.000000,45.000000,52.000000,1.000000,7.656120e+16
50%,252490.000000,4.000000,15.200000,110.000000,237.000000,1.000000,7.656120e+16
75%,346110.000000,7.000000,59.300000,246.000000,828.000000,2.000000,7.656120e+16
max,773900.000000,9.000000,18570.900000,12832.000000,18371.000000,6956.000000,7.656120e+16


In [25]:
reviews_df.dtypes

username                object
product_id               int64
page_order               int64
text                    object
hours                  float64
products               float64
date            datetime64[ns]
early_access              bool
page                     int64
compensation            object
found_funny            float64
user_id                float64
dtype: object

In [26]:
reviews_df.compensation.value_counts()

Product received for free    13286
Name: compensation, dtype: int64

#### Juegos con mas reviews

In [27]:
reviews_df.groupby('product_id').agg({'page_order':'count'}).sort_values('page_order', ascending=False).head()

,page_order
product_id,
440,16570
252490,9161
49520,6349
377160,6335
271590,5239


In [28]:
reviews_df.found_funny.value_counts()

1.0       62816
2.0       17737
3.0        7511
4.0        4259
5.0        2585
          ...  
285.0         1
1153.0        1
1170.0        1
432.0         1
253.0         1
Name: found_funny, Length: 589, dtype: int64

## Ejercicio 2 - Sistema de Recomendación Basado en Usuarios

Este sistema de recomendación deberá entrenar un algoritmo y desarrollar una interfaz que, dado un usuario, le devuelva una lista con los juegos más recomendados.

In [29]:
#cant_horas_user = reviews_df.groupby(['username']).agg( cant_horas = pd.NamedAgg(column='hours', aggfunc=sum))
cant_horas_user = reviews_df.groupby(['username']).hours.sum()
cant_horas_user

username
!                      123.3
!             *          5.6
! 5tryx                  0.9
! DeadlyDeal !           6.3
! Indelible             58.3
                       ...  
󰀈Spectra󰀈                4.9
󰀍 loopuleasa 󰀍          96.9
󰀓Oxymoronicphalanx󰀓      0.4
󰀕СЭНС󰀕                  21.1
󰀗 Lolicage               7.5
Name: hours, Length: 495383, dtype: float64

In [30]:
reviews_df['cant_horas_user'] = reviews_df['username'].apply(lambda x: cant_horas_user[x] 
                                                          if x in cant_horas_user else -1)


In [31]:
reviews_df[ reviews_df['username'] == 'Caesar' ]


,username,product_id,page_order,text,hours,products,date,early_access,page,compensation,found_funny,user_id,cant_horas_user
77242,Caesar,221100,5,"Forever ""Early Access"".",67.0,21.0,2016-02-01,True,2517,None,NaN,7.656120e+16,1723.3
195973,Caesar,327070,0,"Game's unplayable (I know it's EA, but still) ...",1.3,280.0,2016-07-08,True,73,None,NaN,NaN,1723.3
209383,Caesar,214950,4,can't compare to rome 1,686.2,36.0,2014-09-29,False,1081,None,NaN,NaN,1723.3
235587,Caesar,219640,8,"Total loss of money,time and resources.",5.1,88.0,2017-03-27,False,290,None,NaN,7.656120e+16,1723.3
275411,Caesar,204100,0,Good pc port I can max settings @ 1080p with 2...,9.5,48.0,2015-01-31,False,420,None,NaN,NaN,1723.3
285946,Caesar,441790,0,For an EA this game is fun. Graphics are amazi...,81.8,37.0,2016-04-30,True,46,None,NaN,NaN,1723.3
298652,Caesar,588430,7,pay to win.,3.3,76.0,2017-06-20,False,249,None,NaN,NaN,1723.3
349061,Caesar,223850,3,The ultimate pay to win game.\n10/10,19.3,183.0,2017-04-06,False,18,None,3.0,NaN,1723.3
397510,Caesar,582660,8,Incredible design and textures. Game has great...,120.5,92.0,2017-07-04,False,346,None,NaN,7.656120e+16,1723.3
408255,Caesar,204300,8,Awesomenauts is a side-scrolling 3v3 DOTA-styl...,100.3,76.0,2015-05-14,False,545,None,NaN,NaN,1723.3


In [32]:
reviews_df['rating'] = reviews_df['hours'] / reviews_df['cant_horas_user']


In [33]:
reviews_df[ reviews_df['username'] == 'Caesar' ]


,username,product_id,page_order,text,hours,products,date,early_access,page,compensation,found_funny,user_id,cant_horas_user,rating
77242,Caesar,221100,5,"Forever ""Early Access"".",67.0,21.0,2016-02-01,True,2517,None,NaN,7.656120e+16,1723.3,0.038879
195973,Caesar,327070,0,"Game's unplayable (I know it's EA, but still) ...",1.3,280.0,2016-07-08,True,73,None,NaN,NaN,1723.3,0.000754
209383,Caesar,214950,4,can't compare to rome 1,686.2,36.0,2014-09-29,False,1081,None,NaN,NaN,1723.3,0.398190
235587,Caesar,219640,8,"Total loss of money,time and resources.",5.1,88.0,2017-03-27,False,290,None,NaN,7.656120e+16,1723.3,0.002959
275411,Caesar,204100,0,Good pc port I can max settings @ 1080p with 2...,9.5,48.0,2015-01-31,False,420,None,NaN,NaN,1723.3,0.005513
285946,Caesar,441790,0,For an EA this game is fun. Graphics are amazi...,81.8,37.0,2016-04-30,True,46,None,NaN,NaN,1723.3,0.047467
298652,Caesar,588430,7,pay to win.,3.3,76.0,2017-06-20,False,249,None,NaN,NaN,1723.3,0.001915
349061,Caesar,223850,3,The ultimate pay to win game.\n10/10,19.3,183.0,2017-04-06,False,18,None,3.0,NaN,1723.3,0.011199
397510,Caesar,582660,8,Incredible design and textures. Game has great...,120.5,92.0,2017-07-04,False,346,None,NaN,7.656120e+16,1723.3,0.069924
408255,Caesar,204300,8,Awesomenauts is a side-scrolling 3v3 DOTA-styl...,100.3,76.0,2015-05-14,False,545,None,NaN,NaN,1723.3,0.058202


Verificamos que no hayan quedado valores nulos en el rating que calculamos

In [34]:
reviews_df['rating'].isnull().sum()

2616

Vamos a reemplazar los nulos por 0

In [35]:
import numpy as np

reviews_df['rating'] = reviews_df['rating'].replace(to_replace = np.nan, value =0 )

In [36]:
reviews_df['rating'].isnull().sum()

0

In [37]:
import pandas as pd
import seaborn as sns

ax = sns.distplot(reviews_df['rating'])

### Armamos la matriz de usuario-contenido

In [38]:
muc_df = reviews_df[ ['username','product_id','rating'] ]
muc_df.head()

,username,product_id,rating
0,SPejsMan,227940,0.625000
1,Spodermen,270170,0.308176
2,josh,41700,0.052637
3,Sammyrism,332310,0.054054
4,moonmirroir,303210,1.000000


In [39]:
len(reviews_df)

700000

In [40]:
len(muc_df)

700000

In [41]:
muc_df = muc_df.sample(50000)

In [42]:
muc_df.head()

,username,product_id,rating
152141,15xCheetahs,108600,1.00000
119117,Sine,256290,0.20625
646721,Mr Krampus,227300,1.00000
373440,helios.vmg,8190,1.00000
217524,Klershwagen,233840,1.00000


In [43]:
from surprise import Dataset, Reader, KNNWithMeans, BaselineOnly
from surprise.accuracy import rmse
from surprise.model_selection import cross_validate, train_test_split


In [44]:
#ratings_train, ratings_test = train_test_split(ratings, test_size=0.2)

# model = KNNWithMeans(k=3).fit( ratings.build_full_trainset() )

reader = Reader(rating_scale=(muc_df.rating.min(), muc_df.rating.max()))

ratings = Dataset.load_from_df( muc_df[["username", "product_id", "rating"]], reader)

trainset = ratings.build_full_trainset()
                               
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
model = BaselineOnly(bsl_options=bsl_options)

model.fit( trainset )


Estimating biases using als...


### Trabajamos en la funcion de recomendacion

In [70]:
def get_game_name( game_id ):
    return games_df [ games_df['id'] == game_id ]['app_name'].tolist()


El desarrollo de la funcion esta inspirado en el siguiente trabajo:  
https://blog.cambridgespark.com/tutorial-practical-introduction-to-recommender-systems-dbe22848392b


In [75]:
def recomend_games_for_user( user, model, cant_games  ):
    
    games_rated_by_user = []
    games_to_predict_for_user = []
    testset = []
    
    # armamos la lista de los juegos que el usuario rankeo

    games_rated_by_user = muc_df.loc[ muc_df['username'] ==  user, 'product_id']
    
    # calculamos la lista de juegos que tenemos que predecir (i.e. todos - los que el usuario rankeo)
    
    games_to_predict_for_user = np.setdiff1d(games, games_rated_by_user)

    testset = [[user, prod_id, 4.] for prod_id in games_to_predict_for_user]
    
    preds = model.test(testset)

    df = pd.DataFrame(preds, columns=['uid', 'iid', 'rui', 'est', 'details'])

    best_predictions = df.sort_values(by='est', ascending=False)[:cant_games]
    
    rec_games = []
    for game_id in best_predictions.iid:
        rec_games.append( get_game_name(game_id ))
        
    return rec_games, best_predictions


In [76]:
recs = []
recs, best = recomend_games_for_user( 'Lyra', model, 5 )
for r in recs:
    print(r)
best

['Oxygen Not Included']
['Counter-Strike: Global Offensive']
['Infinity Wars: Animated Trading Card Game']
['Magic Duels']
['Terraria']


,uid,iid,rui,est,details
4365,Lyra,457140,4.0,0.924455,{'was_impossible': False}
24,Lyra,730,4.0,0.919938,{'was_impossible': False}
1661,Lyra,257730,4.0,0.904845,{'was_impossible': False}
2540,Lyra,316010,4.0,0.896307,{'was_impossible': False}
786,Lyra,105600,4.0,0.894804,{'was_impossible': False}


In [77]:
recs = []
recs, best = recomend_games_for_user( 'Codes', model, 5 )
for r in recs:
    print(r)
best

['Oxygen Not Included']
['Counter-Strike: Global Offensive']
['Infinity Wars: Animated Trading Card Game']
['Magic Duels']
['Terraria']


,uid,iid,rui,est,details
4364,Codes,457140,4.0,0.932957,{'was_impossible': False}
24,Codes,730,4.0,0.928441,{'was_impossible': False}
1661,Codes,257730,4.0,0.913347,{'was_impossible': False}
2539,Codes,316010,4.0,0.904810,{'was_impossible': False}
786,Codes,105600,4.0,0.903307,{'was_impossible': False}


In [78]:
recs = []
recs, best = recomend_games_for_user( 'Sine', model, 5 )
for r in recs:
    print(r)
best

['Oxygen Not Included']
['Counter-Strike: Global Offensive']
['Infinity Wars: Animated Trading Card Game']
['Magic Duels']
['Terraria']


,uid,iid,rui,est,details
4364,Sine,457140,4.0,0.887770,{'was_impossible': False}
24,Sine,730,4.0,0.883253,{'was_impossible': False}
1660,Sine,257730,4.0,0.868160,{'was_impossible': False}
2539,Sine,316010,4.0,0.859622,{'was_impossible': False}
786,Sine,105600,4.0,0.858119,{'was_impossible': False}


In [52]:
import pandas as pd
import seaborn as sns

ax = sns.distplot( recs['est'])

TypeError: list indices must be integers or slices, not str

## Ejercicio 3 - Sistema de Recomendación Basado en Juegos

Similar al caso anterior, con la diferencia de que este sistema espera como entrada el nombre de un juego y devuelve una lista de juegos similares. El sistema deberá estar programado en base a información de contenido de los juegos (i.e. filtrado basado en contenido o sistema híbrido).

In [ ]:
# Completa